In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path_all = 'G:/DeepLearning/Housing Price Prediction/Data Sets/'
dataset = 'G:/DeepLearning/Housing Price Prediction/Data Sets/data_description.txt'
path_train = 'G:/DeepLearning/Housing Price Prediction/Data Sets/train.csv'
path_test = 'G:/DeepLearning/Housing Price Prediction/Data Sets/test.csv'
house_data = pd.read_csv(path_train)

In [ ]:
house_data.head()

In [ ]:
house_data.info()

In [ ]:
house_data.shape

In [ ]:
print(house_data.columns[house_data.isna().any()].tolist())
len(house_data.columns[house_data.isna().any()].tolist())

In [ ]:
house_data.describe()

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
house_data.hist(bins =50, figsize=(30,25))
plt.show()

In [ ]:
#def split_train_test(data,test_ratio):
   # shuffled_indices = np.random.permutation(len(data))
    #test_set_size = int(len(data)* test_ratio)
    #test_indices = shuffled_indices[:test_set_size]
    #train_indices = shuffled_indices[test_set_size:]
    #return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
#train_set, test_set = split_train_test(dataset, 0.15)
#len(train_set)

In [ ]:
features = [x for x in house_data.columns if x not in ['SalePrice']]
X = house_data[features]
y = house_data['SalePrice']

In [ ]:
corr_matrix = house_data.corr()

In [ ]:
corr_matrix["Id"].sort_values(ascending = False)

In [ ]:
from pandas.plotting import scatter_matrix 
attributes = ["Id","PoolArea","BedroomAbvGr","TotRmsAbvGrd","MoSold","GarageArea",       
"GarageCars","OverallCond","MSSubClass","1stFlrSF","GrLivArea","HalfBath",         
"2ndFlrSF","FullBath","KitchenAbvGr","EnclosedPorch","BsmtFullBath","ScreenPorch","YrSold",           
"GarageYrBlt","OpenPorchSF","BsmtFinSF1","BsmtFinSF2","MiscVal",         
"BsmtUnfSF","LotFrontage","YearBuilt","TotalBsmtSF","Fireplaces","BsmtHalfBath",    
"SalePrice","YearRemodAdd","OverallQual","WoodDeckSF","LotArea","LowQualFinSF",    
"3SsnPorch","MasVnrArea"]
scatter_matrix(house_data[attributes], figsize = (100,100))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)

numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]


numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
])

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(random_state=0)

tree_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('tree_model', tree_model)
                     ])

tree_clf.fit(X_train, y_train)

tree_clf.fit(X_train, y_train)

tree_preds = tree_clf.predict(X_valid)

print('Mean Absolute Error:', mean_absolute_error(y_valid, tree_preds))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

random_model = RandomForestRegressor(random_state=0)

random_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('random_model', random_model)
                     ])

random_clf.fit(X_train, y_train)

random_clf.fit(X_train, y_train)

random_preds = random_clf.predict(X_valid)

print('Mean Absolute Error:', mean_absolute_error(y_valid, random_preds))

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.07, random_state=0)

xgb_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('xgb_model', xgb_model)
                     ])

xgb_clf.fit(X_train, y_train, xgb_model__verbose=False)

xgb_clf.fit(X_train, y_train)

xgb_preds = xgb_clf.predict(X_valid)

print('Mean Absolute Error:', mean_absolute_error(y_valid, xgb_preds))

In [ ]:
hp_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.5, gpu_id=1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=4,
             min_child_weight=1, monotone_constraints='1',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity= 0)

hp_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('hp_model', hp_model)
                     ])

hp_clf.fit(X_train, y_train, hp_model__verbose= 0)

hp_preds = hp_clf.predict(X_valid)

print('Mean Absolute Error:', mean_absolute_error(y_valid, hp_preds))

In [ ]:
X.columns.to_list()

In [ ]:
print(X['YearBuilt'].head())
print(X['YearRemodAdd'].head())

In [ ]:
print(X['LotArea'].head())
print(X['LotFrontage'].head())
print(set(X['LandSlope']))
print(set(X['LandContour']))

In [ ]:
print(set(X['YrSold']))
print(set(X['MoSold']))
print(set(X['Condition1']))
print(set(X['Condition2']))

In [ ]:
print(set(X['ExterQual']))
print(set(X['ExterCond']))

In [ ]:
X_feat_eng = X.copy()
X_feat_eng['years_since_update'] = X_feat_eng['YearRemodAdd'] - X_feat_eng['YearBuilt']
X_feat_eng['geometry'] = X_feat_eng['LotArea'] / X_feat_eng['LotFrontage']
X_feat_eng['land_topology'] = X_feat_eng['LandSlope'] + '_' + X_feat_eng['LandContour']

feature_numerical_cols = [cname for cname in X_feat_eng.columns if 
                X_feat_eng[cname].dtype in ['int64', 'float64']]

feature_categorical_cols = [cname for cname in X_feat_eng.columns if
                    X_feat_eng[cname].nunique() < 13 and 
                    X_feat_eng[cname].dtype == "object"]


feature_numerical_transformer = SimpleImputer(strategy='constant')

feature_categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

feature_preprocessor = ColumnTransformer(
    transformers=[
        ('num', feature_numerical_transformer, feature_numerical_cols),
        ('cat', feature_categorical_transformer, feature_categorical_cols)
])

feature_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.0, gpu_id=1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=4,
             min_child_weight=0.0, monotone_constraints='1',
             n_estimators=1325, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=0)
feature_clf = Pipeline(steps=[('feature_preprocessor', feature_preprocessor),
                      ('feature_model', feature_model)
                     ])

feature_X_train, feature_X_valid, feature_y_train, feature_y_valid = train_test_split(X_feat_eng, y, random_state=0)

feature_clf.fit(feature_X_train, feature_y_train, feature_model__verbose=0)

feature_preds = feature_clf.predict(feature_X_valid)

print('Mean Absolute Error:', mean_absolute_error(feature_y_valid, feature_preds))

In [ ]:
X_test = pd.read_csv(path_test)

In [ ]:
X_test['years_since_update'] = X_test['YearRemodAdd'] - X_test['YearBuilt']
X_test['geometry'] = X_test['LotArea'] / X_test['LotFrontage']
X_test['land_topology'] = X_test['LandSlope'] + '_' + X_test['LandContour']

In [ ]:
preds = feature_clf.predict(X_test)
output = pd.DataFrame({'Id': X_test.Id,
                       'SalePrice': preds})
output.to_csv('G:/DeepLearning/Housing Price Prediction/Data Sets/results.csv', index=False)

In [ ]:
preds = feature_clf.predict(X_test)
output = pd.DataFrame({'Id': X_test.Id,
                       'SalePrice': preds})
output.to_csv('G:/DeepLearning/Housing Price Prediction/Data Sets/submission.csv', index=False)